In [1]:
#kinit command in Python
from subprocess import Popen, PIPE
import getpass
userid=getpass.getuser()
password = getpass.getpass()
kinit = '/usr/bin/kinit'
kinit_args = [ kinit, '%s' % (userid) ]
kinit_args = [ kinit ]
kinit = Popen(kinit_args, stdin=PIPE, stdout=PIPE, stderr=PIPE,universal_newlines=True)
kinit.stdin.write('%s\n' % password)
#kinit.wait()
out,err = kinit.communicate()

········


In [2]:
from bdpcommon.runspark import Runspark 

conf = {'app_name': 'TEST', 'spark.yarn.queue': 'DEV', 'spark.driver.memory': '15g',
        'spark.driver.maxResultSize': '2g',
        'spark.port.maxRetries': 50,
        'spark.executor.instances' : 2, 'spark.executor.memory' : '2g', 'spark.executor.cores': 2}
rp = Runspark(conf=conf, env_name = "cmi-py3_0.1", cluster_name="dev-el3207", spark_version="2.3.0")

sqlContext = rp.sqlCtx
sc = rp.sc
spark = rp.spark


 @       Welcome to  Proximus      __     __  @
 @    ____              __        /  \   /  \ @
 @   / __/__  ___ _____/ /__      \___|_|___/ @
 @  _\ \/ _ \/ _ `/ __/  '_/       ___|_|___  @
 @ /__ / .__/\_,_/_/ /_/\_\       /   | |   \ @
 @    /_/                         \__/   \__/ @
      version 2.3.0.2.6.5.0-292

Using Python version 3.5.5 (default, May 13 2018 21:12:35)
Using environment: cmi-py3_0.1
Spark session available as <runspark_object>.spark,
SparkContext available as <runspark_object>.sc,
SQLContext available as <runspark_object>.sqlCtx.


In [8]:
spark.conf.set("app_name", "Test")

In [9]:
app_name = spark.conf.get('spark.app.name')

driver_tcp_port = spark.conf.get('spark.driver.port')

num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: TEST
Driver TCP port: 45157
Number of partitions: 200


# import data

Import tricks:
* Spark prefer many smaller files than one big
* if not possible read it, then write it to parquet -> the next import will be faster
* can use the * to import multiple files at once

In [1]:
# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2018.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet("AA_DFW_ALL.parquet")

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

# Data manipulation

In [15]:
import pyspark.sql.functions as F

In [ ]:
voter_df.select("VOTER_NAME").distinct().show(40, truncate=False)

voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains("_"))

In [ ]:
voter_df = voter_df.withColumn("splits", F.split(voter_df.VOTER_NAME, '\s+'))

voter_df = voter_df.withColumn("first_name", voter_df.splits.getItem(0))

voter_df = voter_df.withColumn("last_name", voter_df.splits.getItem(F.size('splits') - 1))

voter_df = voter_df.drop('splits')

voter_df.show()

In [ ]:
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == "Mayor", 2)
                               .otherwise(0))

voter_df.show()


In [ ]:
voter_df = df.select(df["VOTER NAME"]).distinct()

print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

In [ ]:
voter_df = voter_df.distinct().cache() # improve performance
voter_df.count()

voter_df.is_cached

vector_df.unpersist()

In [ ]:
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast # improve joins when small datasets

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.join(broadcast(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

## UDF

In [ ]:
def getFirstAndMiddle(names):
  return ' '.join(names)

udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

voter_df = voter_df.drop("first_name")
voter_df = voter_df.drop("splits")
voter_df.show()